In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
from pathlib import Path

from fastai import *
from fastai.vision import *
from fastai.utils.mem import *

# @lukemelas EfficientNet implementation: https://github.com/lukemelas/EfficientNet-PyTorch
#from efficientnet_pytorch import EfficientNet

# Modified version of @lukemelas' EfficientNet implementation with Mish instead of Swish activation
from MEfficientNet_PyTorch.efficientnet_pytorch import EfficientNet as MEfficientNet

# @lessw2020 implementation : https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer/blob/master/ranger.py
# version 9.3.19 used
from ranger import Ranger

In [ ]:
path = 'data/'
labels_df = pd.read_csv('labels_df.csv')
labels_df.head(3)

In [ ]:
def get_data(SZ:int=299):
    SEED = 42
    LABEL = 'class_name'
    
    # Full training set for training, full test set for validation
    src_test = (ImageList.from_df(labels_df, path, folder='merged', cols='filename')
           # the 'is_test' column has values of 1 for the test set
           .split_from_df(col='is_test')
           .label_from_df(cols=LABEL))

    data_test = (src_test.transform(car_tfms, 
                                  size=SZ,  
                                  resize_method=ResizeMethod.SQUISH, 
                                  padding_mode='reflection')
                .databunch()
                .normalize(imagenet_stats))
    
    return data_test

data_test = get_data(SZ=456, do_cutout=False)

In [ ]:
data_test.show_batch()

In [ ]:
from fastai.callbacks import *

def FlatCosAnnealScheduler(learn, lr:float=4e-3, tot_epochs:int=1, moms:Floats=(0.95,0.999),
                          start_pct:float=0.72, curve='cosine'):
    "Manage FCFit trainnig as found in the ImageNette experiments"
    n = len(learn.data.train_dl)
    anneal_start = int(n * tot_epochs * start_pct)
    batch_finish = ((n * tot_epochs) - anneal_start)
    if curve=="cosine":
        curve_type=annealing_cos
    elif curve=="linear":
        curve_type=annealing_linear
    elif curve=="exponential":
        curve_type=annealing_exp
    else:
        raiseValueError(f"annealing type not supported {curve}")

    phase0 = TrainingPhase(anneal_start).schedule_hp('lr', lr).schedule_hp('mom', moms[0])
    phase1 = TrainingPhase(batch_finish).schedule_hp('lr', lr, anneal=curve_type).schedule_hp('mom', moms[1])
    phases = [phase0, phase1]
    return GeneralScheduler(learn, phases)
                
def fit_fc(learn:Learner, tot_epochs:int=None, lr:float=defaults.lr,  moms:Tuple[float,float]=(0.95,0.85), start_pct:float=0.72,
                  wd:float=None, callbacks:Optional[CallbackList]=None, show_curve:bool=False)->None:
    "Fit a model with Flat Cosine Annealing"
    max_lr = learn.lr_range(lr)
    callbacks = listify(callbacks)
    callbacks.append(FlatCosAnnealScheduler(learn, lr, moms=moms, start_pct=start_pct, tot_epochs=tot_epochs))
    learn.fit(tot_epochs, max_lr, wd=wd, callbacks=callbacks)

In [ ]:
effnet_b7 = 'efficientnet-b7'
def getMishModel(data, model_name):
    model = MEfficientNet.from_pretrained(model_name, data.c)
    return model

In [ ]:
mish_model = getMishModel(data_test, effnet_b7) 

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from ranger import Ranger

data_test.batch_size = 12

learn = Learner(data_test, 
                model=mish_model,
                wd = 1e-3,
                opt_func=Ranger,
                bn_wd=False,
                true_wd=True,
                metrics=[accuracy],
                loss_func=LabelSmoothingCrossEntropy(),
                # callback_fns=BnFreeze
               ).to_fp16()

fit_fc(learn, tot_epochs=40, lr=15e-4, start_pct=0.1, wd=1e-3, show_curve=False)

# SZ 456 - 5 - lr: 15e-4, start_pct=0.10, wd1e-3